In [ ]:
import openai
import os
from pathlib import Path  
import json
import os 
from pprint import pprint
import requests
from azure.search.documents.models import Vector  
import uuid
from tenacity import retry, wait_random_exponential, stop_after_attempt  

from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from openai.embeddings_utils import get_embedding, cosine_similarity
import inspect
env_path = Path('.') / 'secrets.env'
load_dotenv(dotenv_path=env_path)

In [ ]:
from langchain.utilities import BingSearchAPIWrapper

def search_knowledgebase(search_query, k=5):
    """
    Searches the knowledge base for an answer to the question.

    Args:
        search_query (str): The search query to use to search the knowledge base.

    Returns:
        str: The answer to the question.
    """

    # Add your Bing Search V7 subscription key and endpoint to your environment variables.
    search = BingSearchAPIWrapper(k=k)

    #limit to docs.confluent.io
    search_query = search_query +" site:docs.confluent.io"

    return search.run(search_query)


In [ ]:
query = "how to query the list of clusters in confluent cloud" 

search_knowledgebase(query)

In [ ]:
import base64
def execute_graphql(graphql_query):
    """
    Execute a graphql query on customer's environment

    Args:
        graphql_query (str): syntactically valid graphql query to retrieve information from customer's environment

    Returns:
        str: The result of the graphql query.
    """

    api_key = os.getenv("CF_API_KEY")
    api_secret = os.getenv("CF_API_SECRET")
    graphql_endpoint = os.getenv("CF_GRAPHQL_ENDPOINT")

    # Concatenate the API key and secret and encode them in Base64
    credentials = f"{api_key}:{api_secret}"
    encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')
    print(encoded_credentials)
    # Set up the headers with the encoded credentials
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Basic {encoded_credentials}'
    }

    # Perform the HTTP POST request
    response = requests.post(graphql_endpoint, headers=headers, json={'query': graphql_query})

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        result = response.json()
        return result
    else:
        return f"Request failed with status code {response.status_code}: {response.text}"


In [ ]:
graphql_query = """{
  __schema {
    queryType {
      fields {
        name
        description
        
      }
    }
  }
}
"""
execute_graphql(graphql_query)

In [ ]:
def get_schema(graphql_type):
    """
    Retrieve detail schema of a graphql type

    Args:
        graphql_type (str): name of the graphql type to retrieve schema

    Returns:
        str: The schema of the graphql type.
    """

    graph_query = f"""
{{
  __type(name: "{graphql_type}") {{
    name
    kind
    description
    fields {{
      name
      description
      type {{
        name
        kind
      }}
    }}
  }}
}}
"""
    return execute_graphql(graph_query)


In [ ]:
graphql_type= "kafka_topic"
get_schema(graphql_type)